In [1]:
import pandas as pd
from glob import glob
import re
import matplotlib.pyplot as plt
import numpy as np

In [290]:
pd.set_option('display.max_colwidth',100)

In [3]:
src_data = r'c:\Users\jaromir\OneDrive\UoM\100_Disertation\02_SrcData'

In [151]:
filenames = glob(src_data+'\$*.csv')
df = pd.DataFrame()

for file in filenames:
    # read a csv from source folder
    data = pd.read_csv(file, encoding = 'utf-8')
    # parese the name of the file to extract the ticker name and save it 
    # as a new attribute
    data['ticker'] = re.search(r'\$[A-Z]+_',file).group(0)[1:-1]
    # concatenate the csv with the rest of the DataFrame
    df = pd.concat([df, data], axis=0, sort=False, ignore_index = True)
    df.drop(labels = 'Unnamed: 0', inplace=True, axis=1)

In [5]:
# Load fiannce data
stock_price = pd.read_csv(src_data+'/stock_price.csv')

In [6]:
# Tokenization
from nltk.tokenize import sent_tokenize, word_tokenize, TweetTokenizer
from nltk.tokenize.casual import TweetTokenizer as CasualTweetTokenizer
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import WordNetLemmatizer
from functools import reduce
from collections import Counter

In [267]:
# POS tagging
import nltk
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\jaromir\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [152]:
df.shape

(50000, 16)

In [153]:
# create a copy with fewer attributes
df_text = df[['text','ticker']]

In [154]:
df_text.head(3)

,text,ticker
0,$AMZN $GOOG $AAPL $MSFT Google is the best.... $NKLA $NFLX $DIS $SPX $FB $UBER $LYFT $COST $SPY $DAL $UAL $LIND $NKE $DJI $AMD $GE $BAC $CCL $RCL $NCL $SAVE $TSLA $OIL $BTC $AAL $F $BA $MRO $OILU $BAC $WFC $NKE $WMT $CLX $PTON $CSCO $WMT $ZM $GILD $M $NIO $MGM $COTY $ABEV,DIS
1,A lot of people ask me what I’m currently in and I’ll be Transparent (goes in order by Portfolio %) $AIM $IDEX $FCEL $VBIV $MARK $LUNA Options in; $DIS $NKLA $CEMI That’s my portfolio. If you have questions feel free to dm me!,DIS
2,"Together Everyone Achieves More‼️LIVE! FREE day trade alerts, swing trade alerts, as well as watchlists. Join us! 👇🏼 http://discord.gg/T9CxYEM $TSLA $SPY $ACB $SPCE $AAPL $UGAZ $MFA $WLL $AMD $NKLA $BA $WTRH $OAS $TVIX $MRNA $HTZ $BYND $JNUG $DIS $CCL $AAL",DIS


In [155]:
# make a subsample for faster handling
# select a random sample of 5 tweets for each ticker
smpl = df_text.groupby('ticker').apply(lambda x: x.sample(n=5, random_state = 10))

# get rid of multi-level index
smpl.reset_index(drop=True, inplace=True)

# print the results
smpl

,text,ticker
0,Market cap is already higher than $dis paid for Jim Henson’s muppets but sure buy out a company because it has a licensing deal with Llama llama,DIS
1,$DIS what a move!!!! 💪🏻 all out at $104.95 target. Now wait until $95 to play again $SPY $MSFT $AMZN $FB $SBUX $JPM $CMG $JNJ $BABA $NKE $CCL $UNH $NVDA $DIS $VZ $TSLA $QQQ $MSI $WMT $NOW $NFLX $DECK $MELI $BKNG $PLNT $BA $CCL $WYNN $RCL $AAL,DIS
2,$T $DIS $AMC - California sets up movie theater reopenings https://seekingalpha.com/news/3581360-california-sets-up-movie-theater-reopenings?source=feed_f&amp;utm_campaign=twitter_automated&amp;utm_content=news&amp;utm_medium=social&amp;utm_source=twitter_automated,DIS
3,LONG 100 $JNJ 70 $XOM 50 $V 100 $BAC 60 $MA 100 $DIS 100 $C 100 $ADBE 100 $PYPL 100 $CRM 100 $AVGO 100 $ABBV 100 $ACN 100 $LLY 50 $IWM 60 $EEM 100 $SLV 100 $BKLN 100 $XLV 90 $XLY 100 $DB 100 $M 60 $TEVA,DIS
4,"I don't like big dollar stocks, but I'm watching $DIS for a possible short term entry. With opening of Disney Springs this week, the parks are likely going to be opening soon for phase 1.",DIS
5,$MSIM Family taking over net worth $500 million. Huh?💸📈 #STOCKS #blockchain #Crypto #bitcoin #XRP #ico #startup $TWTR $DWDP $PNC $CVS $BABA $NIO $TGT $MER $MGM $TLRY $CHK $AMZN $COST $LULU $YUM $VRUS $LDSR $FB $SQ $VALE $YELP $TE $PSSI $JD $BZUN $XOM $BP $C $S $F $jpm $nvda $v,JPM
6,"Really hard times are ahead! Your fav restaurant and coffee shop will close, kids won't go back to school, someone you know might die from virus! This is such a big deal! Oil drillers will go bankrupt and gas will go up in long term. Really bad days ahead! $DJI $SPY $XOM $JPM $GM",JPM
7,$MISM GOING! UNNOTICED silent R/MERGER!💸📈🛒✈🍾 $AAPL $AMD $AMRN $AAL $BAC $AMZN $MSFT $BA $ABT $TSLA $X $NVDA $GE $DIS $CCL $UBER $DAL $NFLX $FB $T $F $MU $SQ $M $ZM $XOM $GILD $ROKU $UAL $JPM $ABBV $FCX $SNAP $BABA $WFC $RCL $BB $HSBC $LYG $TEVA $C $OXY $JNJ $LYFT $LK $ZM,JPM
8,"GD Entertainment and Technology sets relaunch of re-imagined prepaid ‘Dream Card' Self-Use, Millennials and Gen Z consumers will drive prepaid gift cards to 9.2% CAGR growth from 2020-2024 https://bit.ly/2Cc2DeN #millennials #growth #dreamcard $GDET $GDOT $AXP $PYPL $JPM",JPM
9,Here's what the traders are eyeing heading into the weekend. $JPM $GDX $AAPL $AGN,JPM


In [156]:
smpl.loc[2,'text']

'$T $DIS $AMC - California sets up movie theater reopenings https://seekingalpha.com/news/3581360-california-sets-up-movie-theater-reopenings?source=feed_f&amp;utm_campaign=twitter_automated&amp;utm_content=news&amp;utm_medium=social&amp;utm_source=twitter_automated'

In [157]:
# remove duplicates
smpl.drop_duplicates(subset ="text", keep = 'first', inplace = True)
smpl.reset_index(drop=True, inplace=True)

### Remove unwanted strings and words

In [158]:
# remove urls
url_filter = re.compile(r"www.[\w\d]+.\w+|http://\S+|https://\S+")
re.sub(url_filter, "",smpl.loc[22,'text'])

'#Options Flow Grid Update $XOM #OptionsTrading Exxon Mobil Shares ⬆️0.72% to $46.24 #OptionsFlow #stocks #stock #StockMarket #investing #Investment #energy #Oil #oilandgas #OilPrices #oilprice  '

In [159]:
# remove hashtags
hashtag_filter = re.compile(r"#\w+")
re.sub(hashtag_filter, "",smpl.loc[22,'text'])

' Flow Grid Update $XOM  Exxon Mobil Shares ⬆️0.72% to $46.24            https://apple.co/2XZuTYw https://twitter.com/MinteractApp/status/1265774020248641537?s=20'

In [213]:
# remove mentions
mentions_filter = re.compile(r"@\w+")
re.sub(mentions_filter, "",smpl.loc[1,'text'])

'$DIS what a move!!!! 💪🏻 all out at $104.95 target. Now wait until $95 to play again $SPY $MSFT $AMZN $FB $SBUX $JPM $CMG $JNJ $BABA $NKE $CCL $UNH $NVDA $DIS $VZ $TSLA $QQQ $MSI $WMT $NOW $NFLX $DECK $MELI $BKNG $PLNT $BA $CCL $WYNN $RCL $AAL'

In [215]:
# remove numbers, amounts and dates
number_filter = re.compile(r"""\$[0-9.,]+           # $46.24
                               |-*[0-9.,]+%         # 0.72%, -1.4%
                               |[+-]*\d+            # +71
                                """,
                                re.VERBOSE)
#re.findall(number_filter, smpl.loc[1,'text'])
re.sub(number_filter, " ",smpl.loc[22,'text'])

'#Options Flow Grid Update $XOM #OptionsTrading Exxon Mobil Shares ⬆️  to   #OptionsFlow #stocks #stock #StockMarket #investing #Investment #energy #Oil #oilandgas #OilPrices #oilprice https://apple.co/ XZuTYw https://twitter.com/MinteractApp/status/ ?s= '

In [216]:
# remove special characters
schar_filter = re.compile(r"[+?!,;:.(){}-]")
re.sub(schar_filter, "",smpl.loc[13,'text'])

'CocaCola $KO is currently 14% at $4560 very reflective of the Dow L/t bullish high sugar and caffeine carbonated drinks @JoePiSkates'

In [217]:
def filter_results(document, filter_condition):
    """replace given regEx string with a space"""
    document = re.sub(filter_condition, " ", document)
    return document

In [218]:
filters = [url_filter, hashtag_filter, mentions_filter, number_filter, schar_filter]

In [219]:
def filter_documents(df, filters):
    """filter results for all documents"""
    df.loc[:,'filtered_text'] = df.loc[:,'text']
    for char_filter in filters:       
        df.loc[:,'filtered_text'] = df.loc[:,'filtered_text'].apply(filter_results, args=[char_filter])
    return df

In [220]:
smpl = filter_documents(smpl, filters)

In [222]:
smpl.loc[:5,['text','filtered_text']]

,text,filtered_text
0,Market cap is already higher than $dis paid for Jim Henson’s muppets but sure buy out a company because it has a licensing deal with Llama llama,Market cap is already higher than $dis paid for Jim Henson’s muppets but sure buy out a company because it has a licensing deal with Llama llama
1,$DIS what a move!!!! 💪🏻 all out at $104.95 target. Now wait until $95 to play again $SPY $MSFT $AMZN $FB $SBUX $JPM $CMG $JNJ $BABA $NKE $CCL $UNH $NVDA $DIS $VZ $TSLA $QQQ $MSI $WMT $NOW $NFLX $DECK $MELI $BKNG $PLNT $BA $CCL $WYNN $RCL $AAL,$DIS what a move 💪🏻 all out at target Now wait until to play again $SPY $MSFT $AMZN $FB $SBUX $JPM $CMG $JNJ $BABA $NKE $CCL $UNH $NVDA $DIS $VZ $TSLA $QQQ $MSI $WMT $NOW $NFLX $DECK $MELI $BKNG $PLNT $BA $CCL $WYNN $RCL $AAL
2,$T $DIS $AMC - California sets up movie theater reopenings https://seekingalpha.com/news/3581360-california-sets-up-movie-theater-reopenings?source=feed_f&amp;utm_campaign=twitter_automated&amp;utm_content=news&amp;utm_medium=social&amp;utm_source=twitter_automated,$T $DIS $AMC California sets up movie theater reopenings
3,LONG 100 $JNJ 70 $XOM 50 $V 100 $BAC 60 $MA 100 $DIS 100 $C 100 $ADBE 100 $PYPL 100 $CRM 100 $AVGO 100 $ABBV 100 $ACN 100 $LLY 50 $IWM 60 $EEM 100 $SLV 100 $BKLN 100 $XLV 90 $XLY 100 $DB 100 $M 60 $TEVA,LONG $JNJ $XOM $V $BAC $MA $DIS $C $ADBE $PYPL $CRM $AVGO $ABBV $ACN $LLY $IWM $EEM $SLV $BKLN $XLV $XLY $DB $M $TEVA
4,"I don't like big dollar stocks, but I'm watching $DIS for a possible short term entry. With opening of Disney Springs this week, the parks are likely going to be opening soon for phase 1.",I don't like big dollar stocks but I'm watching $DIS for a possible short term entry With opening of Disney Springs this week the parks are likely going to be opening soon for phase
5,$MSIM Family taking over net worth $500 million. Huh?💸📈 #STOCKS #blockchain #Crypto #bitcoin #XRP #ico #startup $TWTR $DWDP $PNC $CVS $BABA $NIO $TGT $MER $MGM $TLRY $CHK $AMZN $COST $LULU $YUM $VRUS $LDSR $FB $SQ $VALE $YELP $TE $PSSI $JD $BZUN $XOM $BP $C $S $F $jpm $nvda $v,$MSIM Family taking over net worth million Huh 💸📈 $TWTR $DWDP $PNC $CVS $BABA $NIO $TGT $MER $MGM $TLRY $CHK $AMZN $COST $LULU $YUM $VRUS $LDSR $FB $SQ $VALE $YELP $TE $PSSI $JD $BZUN $XOM $BP $C $S $F $jpm $nvda $v


### Handle cashtags

In [223]:
# change $ticker to #ticker so that it get tokenized together
ticker_replacement = re.compile(r"\$\w+")

In [224]:
def ticker_handle(df, sub_regex):
    """finds tickers labeled with $ and replace the dollar sign with # so that it doesnt get parsed
    during tokenization"""
    for idx in range(df.shape[0]):
        ticks = re.findall(sub_regex, df.loc[:,'filtered_text'][idx])
        for tick in ticks:
            df.loc[:,'filtered_text'][idx] = df.loc[:,'filtered_text'][idx].replace(tick, "#"+tick[1:])
    return df

In [225]:
smpl = ticker_handle(smpl, ticker_replacement)
smpl[['text','filtered_text']]

,text,filtered_text
0,Market cap is already higher than $dis paid for Jim Henson’s muppets but sure buy out a company because it has a licensing deal with Llama llama,Market cap is already higher than #dis paid for Jim Henson’s muppets but sure buy out a company because it has a licensing deal with Llama llama
1,$DIS what a move!!!! 💪🏻 all out at $104.95 target. Now wait until $95 to play again $SPY $MSFT $AMZN $FB $SBUX $JPM $CMG $JNJ $BABA $NKE $CCL $UNH $NVDA $DIS $VZ $TSLA $QQQ $MSI $WMT $NOW $NFLX $DECK $MELI $BKNG $PLNT $BA $CCL $WYNN $RCL $AAL,#DIS what a move 💪🏻 all out at target Now wait until to play again #SPY #MSFT #AMZN #FB #SBUX #JPM #CMG #JNJ #BABA #NKE #CCL #UNH #NVDA #DIS #VZ #TSLA #QQQ #MSI #WMT #NOW #NFLX #DECK #MELI #BKNG #PLNT #BA #CCL #WYNN #RCL #AAL
2,$T $DIS $AMC - California sets up movie theater reopenings https://seekingalpha.com/news/3581360-california-sets-up-movie-theater-reopenings?source=feed_f&amp;utm_campaign=twitter_automated&amp;utm_content=news&amp;utm_medium=social&amp;utm_source=twitter_automated,#T #DIS #AMC California sets up movie theater reopenings
3,LONG 100 $JNJ 70 $XOM 50 $V 100 $BAC 60 $MA 100 $DIS 100 $C 100 $ADBE 100 $PYPL 100 $CRM 100 $AVGO 100 $ABBV 100 $ACN 100 $LLY 50 $IWM 60 $EEM 100 $SLV 100 $BKLN 100 $XLV 90 $XLY 100 $DB 100 $M 60 $TEVA,LONG #JNJ #XOM #V #BAC #MA #DIS #C #ADBE #PYPL #CRM #AVGO #ABBV #ACN #LLY #IWM #EEM #SLV #BKLN #XLV #XLY #DB #M #TEVA
4,"I don't like big dollar stocks, but I'm watching $DIS for a possible short term entry. With opening of Disney Springs this week, the parks are likely going to be opening soon for phase 1.",I don't like big dollar stocks but I'm watching #DIS for a possible short term entry With opening of Disney Springs this week the parks are likely going to be opening soon for phase
5,$MSIM Family taking over net worth $500 million. Huh?💸📈 #STOCKS #blockchain #Crypto #bitcoin #XRP #ico #startup $TWTR $DWDP $PNC $CVS $BABA $NIO $TGT $MER $MGM $TLRY $CHK $AMZN $COST $LULU $YUM $VRUS $LDSR $FB $SQ $VALE $YELP $TE $PSSI $JD $BZUN $XOM $BP $C $S $F $jpm $nvda $v,#MSIM Family taking over net worth million Huh 💸📈 #TWTR #DWDP #PNC #CVS #BABA #NIO #TGT #MER #MGM #TLRY #CHK #AMZN #COST #LULU #YUM #VRUS #LDSR #FB #SQ #VALE #YELP #TE #PSSI #JD #BZUN #XOM #BP #C #S #F #jpm #nvda #v
6,"Really hard times are ahead! Your fav restaurant and coffee shop will close, kids won't go back to school, someone you know might die from virus! This is such a big deal! Oil drillers will go bankrupt and gas will go up in long term. Really bad days ahead! $DJI $SPY $XOM $JPM $GM",Really hard times are ahead Your fav restaurant and coffee shop will close kids won't go back to school someone you know might die from virus This is such a big deal Oil drillers will go bankrupt and gas will go up in long term Really bad days ahead #DJI #SPY #XOM #JPM #GM
7,$MISM GOING! UNNOTICED silent R/MERGER!💸📈🛒✈🍾 $AAPL $AMD $AMRN $AAL $BAC $AMZN $MSFT $BA $ABT $TSLA $X $NVDA $GE $DIS $CCL $UBER $DAL $NFLX $FB $T $F $MU $SQ $M $ZM $XOM $GILD $ROKU $UAL $JPM $ABBV $FCX $SNAP $BABA $WFC $RCL $BB $HSBC $LYG $TEVA $C $OXY $JNJ $LYFT $LK $ZM,#MISM GOING UNNOTICED silent R/MERGER 💸📈🛒✈🍾 #AAPL #AMD #AMRN #AAL #BAC #AMZN #MSFT #BA #ABT #TSLA #X #NVDA #GE #DIS #CCL #UBER #DAL #NFLX #FB #T #F #MU #SQ #M #ZM #XOM #GILD #ROKU #UAL #JPM #ABBV #FCX #SNAP #BABA #WFC #RCL #BB #HSBC #LYG #TEVA #C #OXY #JNJ #LYFT #LK #ZM
8,"GD Entertainment and Technology sets relaunch of re-imagined prepaid ‘Dream Card' Self-Use, Millennials and Gen Z consumers will drive prepaid gift cards to 9.2% CAGR growth from 2020-2024 https://bit.ly/2Cc2DeN #millennials #growth #dreamcard $GDET $GDOT $AXP $PYPL $JPM",GD Entertainment and Technology sets relaunch of re imagined prepaid ‘Dream Card' Self Use Millennials and Gen Z consumers will drive prepaid gift cards to CAGR growth from #GDET #GDOT #AXP #PYPL #JPM
9,Here's what the traders are eyeing heading into the weekend. $JPM $GDX $AAPL $AGN,Here's what the traders are eyeing

### Tokenization

In [252]:
# Sentence Tokenization
smpl.loc[:,'Sent_Token'] = smpl.filtered_text.apply(lambda x: sent_tokenize(x))

In [226]:
# Word Tokenization
smpl.loc[:,'Word_Token'] = smpl.filtered_text.apply(lambda x: TweetTokenizer().tokenize(x))

In [227]:
smpl[['text','Word_Token']].head()

,text,Word_Token
0,Market cap is already higher than $dis paid for Jim Henson’s muppets but sure buy out a company because it has a licensing deal with Llama llama,"[Market, cap, is, already, higher, than, #dis, paid, for, Jim, Henson, ’, s, muppets, but, sure, buy, out, a, company, because, it, has, a, licensing, deal, with, Llama, llama]"
1,$DIS what a move!!!! 💪🏻 all out at $104.95 target. Now wait until $95 to play again $SPY $MSFT $AMZN $FB $SBUX $JPM $CMG $JNJ $BABA $NKE $CCL $UNH $NVDA $DIS $VZ $TSLA $QQQ $MSI $WMT $NOW $NFLX $DECK $MELI $BKNG $PLNT $BA $CCL $WYNN $RCL $AAL,"[#DIS, what, a, move, 💪, 🏻, all, out, at, target, Now, wait, until, to, play, again, #SPY, #MSFT, #AMZN, #FB, #SBUX, #JPM, #CMG, #JNJ, #BABA, #NKE, #CCL, #UNH, #NVDA, #DIS, #VZ, #TSLA, #QQQ, #MSI, #WMT, #NOW, #NFLX, #DECK, #MELI, #BKNG, #PLNT, #BA, #CCL, #WYNN, #RCL, #AAL]"
2,$T $DIS $AMC - California sets up movie theater reopenings https://seekingalpha.com/news/3581360-california-sets-up-movie-theater-reopenings?source=feed_f&amp;utm_campaign=twitter_automated&amp;utm_content=news&amp;utm_medium=social&amp;utm_source=twitter_automated,"[#, T, #DIS, #AMC, California, sets, up, movie, theater, reopenings]"
3,LONG 100 $JNJ 70 $XOM 50 $V 100 $BAC 60 $MA 100 $DIS 100 $C 100 $ADBE 100 $PYPL 100 $CRM 100 $AVGO 100 $ABBV 100 $ACN 100 $LLY 50 $IWM 60 $EEM 100 $SLV 100 $BKLN 100 $XLV 90 $XLY 100 $DB 100 $M 60 $TEVA,"[LONG, #JNJ, #XOM, #, V, #BAC, #MA, #DIS, #, C, #ADBE, #PYPL, #CRM, #AVGO, #ABBV, #ACN, #LLY, #IWM, #EEM, #SLV, #BKLN, #XLV, #XLY, #DB, #, M, #TEVA]"
4,"I don't like big dollar stocks, but I'm watching $DIS for a possible short term entry. With opening of Disney Springs this week, the parks are likely going to be opening soon for phase 1.","[I, don't, like, big, dollar, stocks, but, I'm, watching, #DIS, for, a, possible, short, term, entry, With, opening, of, Disney, Springs, this, week, the, parks, are, likely, going, to, be, opening, soon, for, phase]"


In [228]:
# Turn everything lower case
def lower_list(list_of_word):
    """ turn all strings into lower case"""
    low_list = [word.lower() for word in list_of_word]
    return low_list

In [229]:
smpl.loc[:,'Word_Token'] = smpl.loc[:,'Word_Token'].apply(lower_list)

In [230]:
# Get rid of stop words
def remove_stopwords(list_of_word):
    """ removes stopwords """
    no_stopwords = [t for t in list_of_word if t not in stopwords.words('english')]
    return no_stopwords

In [231]:
smpl.loc[:,'Word_Token_NS'] = smpl.loc[:,'Word_Token'].apply(remove_stopwords)

In [232]:
smpl[['text','Word_Token_NS']]

,text,Word_Token_NS
0,Market cap is already higher than $dis paid for Jim Henson’s muppets but sure buy out a company because it has a licensing deal with Llama llama,"[market, cap, already, higher, #dis, paid, jim, henson, ’, muppets, sure, buy, company, licensing, deal, llama, llama]"
1,$DIS what a move!!!! 💪🏻 all out at $104.95 target. Now wait until $95 to play again $SPY $MSFT $AMZN $FB $SBUX $JPM $CMG $JNJ $BABA $NKE $CCL $UNH $NVDA $DIS $VZ $TSLA $QQQ $MSI $WMT $NOW $NFLX $DECK $MELI $BKNG $PLNT $BA $CCL $WYNN $RCL $AAL,"[#dis, move, 💪, 🏻, target, wait, play, #spy, #msft, #amzn, #fb, #sbux, #jpm, #cmg, #jnj, #baba, #nke, #ccl, #unh, #nvda, #dis, #vz, #tsla, #qqq, #msi, #wmt, #now, #nflx, #deck, #meli, #bkng, #plnt, #ba, #ccl, #wynn, #rcl, #aal]"
2,$T $DIS $AMC - California sets up movie theater reopenings https://seekingalpha.com/news/3581360-california-sets-up-movie-theater-reopenings?source=feed_f&amp;utm_campaign=twitter_automated&amp;utm_content=news&amp;utm_medium=social&amp;utm_source=twitter_automated,"[#, #dis, #amc, california, sets, movie, theater, reopenings]"
3,LONG 100 $JNJ 70 $XOM 50 $V 100 $BAC 60 $MA 100 $DIS 100 $C 100 $ADBE 100 $PYPL 100 $CRM 100 $AVGO 100 $ABBV 100 $ACN 100 $LLY 50 $IWM 60 $EEM 100 $SLV 100 $BKLN 100 $XLV 90 $XLY 100 $DB 100 $M 60 $TEVA,"[long, #jnj, #xom, #, v, #bac, #ma, #dis, #, c, #adbe, #pypl, #crm, #avgo, #abbv, #acn, #lly, #iwm, #eem, #slv, #bkln, #xlv, #xly, #db, #, #teva]"
4,"I don't like big dollar stocks, but I'm watching $DIS for a possible short term entry. With opening of Disney Springs this week, the parks are likely going to be opening soon for phase 1.","[like, big, dollar, stocks, i'm, watching, #dis, possible, short, term, entry, opening, disney, springs, week, parks, likely, going, opening, soon, phase]"
5,$MSIM Family taking over net worth $500 million. Huh?💸📈 #STOCKS #blockchain #Crypto #bitcoin #XRP #ico #startup $TWTR $DWDP $PNC $CVS $BABA $NIO $TGT $MER $MGM $TLRY $CHK $AMZN $COST $LULU $YUM $VRUS $LDSR $FB $SQ $VALE $YELP $TE $PSSI $JD $BZUN $XOM $BP $C $S $F $jpm $nvda $v,"[#msim, family, taking, net, worth, million, huh, 💸, 📈, #twtr, #dwdp, #pnc, #cvs, #baba, #nio, #tgt, #mer, #mgm, #tlry, #chk, #amzn, #cost, #lulu, #yum, #vrus, #ldsr, #fb, #sq, #vale, #yelp, #te, #pssi, #jd, #bzun, #xom, #bp, #, c, #, #, f, #jpm, #nvda, #, v]"
6,"Really hard times are ahead! Your fav restaurant and coffee shop will close, kids won't go back to school, someone you know might die from virus! This is such a big deal! Oil drillers will go bankrupt and gas will go up in long term. Really bad days ahead! $DJI $SPY $XOM $JPM $GM","[really, hard, times, ahead, fav, restaurant, coffee, shop, close, kids, go, back, school, someone, know, might, die, virus, big, deal, oil, drillers, go, bankrupt, gas, go, long, term, really, bad, days, ahead, #dji, #spy, #xom, #jpm, #gm]"
7,$MISM GOING! UNNOTICED silent R/MERGER!💸📈🛒✈🍾 $AAPL $AMD $AMRN $AAL $BAC $AMZN $MSFT $BA $ABT $TSLA $X $NVDA $GE $DIS $CCL $UBER $DAL $NFLX $FB $T $F $MU $SQ $M $ZM $XOM $GILD $ROKU $UAL $JPM $ABBV $FCX $SNAP $BABA $WFC $RCL $BB $HSBC $LYG $TEVA $C $OXY $JNJ $LYFT $LK $ZM,"[#mism, going, unnoticed, silent, r, /, merger, 💸, 📈, 🛒, ✈, 🍾, #aapl, #amd, #amrn, #aal, #bac, #amzn, #msft, #ba, #abt, #tsla, #, x, #nvda, #ge, #dis, #ccl, #uber, #dal, #nflx, #fb, #, #, f, #mu, #sq, #, #zm, #xom, #gild, #roku, #ual, #jpm, #abbv, #fcx, #snap, #baba, #wfc, #rcl, #bb, #hsbc, #lyg, #teva, #, c, #oxy, #jnj, #lyft, #lk, #zm]"
8,"GD Entertainment and Technology sets relaunch of re-imagined prepaid ‘Dream Card' Self-Use, Millennials and Gen Z consumers will drive prepaid gift cards to 9.2% CAGR growth from 2020-2024 https://bit.ly/2Cc2DeN #millennials #growth #dreamcard $GDET $GDOT $AXP $PYPL $JPM","[gd, entertainment, technology, sets, relaunch, imagined, prepaid, ‘, dream, card, ', self, use, millennials, gen, z, consumers, drive, prepaid, gift, cards, cagr, growth, #gdet, #gdot, #axp, #pypl, #jpm]"
9,Here's what the traders are eyei

### POS tagging & Lemmatization

In [268]:
def do_pos_tag(list_of_words):
    return nltk.pos_tag(list_of_words)

In [269]:
smpl.loc[:,'POS'] = smpl.loc[:,'Word_Token_NS'].apply(do_pos_tag)

In [270]:
smpl[['text','POS']]

,text,POS
0,Market cap is already higher than $dis paid for Jim Henson’s muppets but sure buy out a company because it has a licensing deal with Llama llama,"[(market, NN), (cap, NN), (already, RB), (higher, JJR), (#dis, NN), (paid, VBN), (jim, NN), (henson, NN), (’, NNP), (muppets, VBZ), (sure, JJ), (buy, VB), (company, NN), (licensing, VBG), (deal, NN), (llama, NN), (llama, NN)]"
1,$DIS what a move!!!! 💪🏻 all out at $104.95 target. Now wait until $95 to play again $SPY $MSFT $AMZN $FB $SBUX $JPM $CMG $JNJ $BABA $NKE $CCL $UNH $NVDA $DIS $VZ $TSLA $QQQ $MSI $WMT $NOW $NFLX $DECK $MELI $BKNG $PLNT $BA $CCL $WYNN $RCL $AAL,"[(#dis, JJ), (move, NN), (💪, NNP), (🏻, NNP), (target, NN), (wait, NN), (play, NN), (#spy, NNP), (#msft, NNP), (#amzn, NNP), (#fb, NNP), (#sbux, NNP), (#jpm, NNP), (#cmg, NNP), (#jnj, NNP), (#baba, NNP), (#nke, NNP), (#ccl, NNP), (#unh, NNP), (#nvda, NNP), (#dis, NNP), (#vz, NNP), (#tsla, NNP), (#qqq, NNP), (#msi, NNP), (#wmt, NNP), (#now, NNP), (#nflx, NNP), (#deck, NNP), (#meli, NNP), (#bkng, NNP), (#plnt, NNP), (#ba, NNP), (#ccl, NNP), (#wynn, NNP), (#rcl, NNP), (#aal, NN)]"
2,$T $DIS $AMC - California sets up movie theater reopenings https://seekingalpha.com/news/3581360-california-sets-up-movie-theater-reopenings?source=feed_f&amp;utm_campaign=twitter_automated&amp;utm_content=news&amp;utm_medium=social&amp;utm_source=twitter_automated,"[(#, #), (#dis, NNP), (#amc, NNP), (california, NN), (sets, NNS), (movie, NN), (theater, NN), (reopenings, NNS)]"
3,LONG 100 $JNJ 70 $XOM 50 $V 100 $BAC 60 $MA 100 $DIS 100 $C 100 $ADBE 100 $PYPL 100 $CRM 100 $AVGO 100 $ABBV 100 $ACN 100 $LLY 50 $IWM 60 $EEM 100 $SLV 100 $BKLN 100 $XLV 90 $XLY 100 $DB 100 $M 60 $TEVA,"[(long, RB), (#jnj, JJ), (#xom, NNP), (#, #), (v, NN), (#bac, NNP), (#ma, NNP), (#dis, NNP), (#, #), (c, NN), (#adbe, NNP), (#pypl, NNP), (#crm, NNP), (#avgo, NNP), (#abbv, NNP), (#acn, NNP), (#lly, RB), (#iwm, VBZ), (#eem, JJ), (#slv, NNP), (#bkln, NNP), (#xlv, NNP), (#xly, NNP), (#db, NNP), (#, #), (#teva, NN)]"
4,"I don't like big dollar stocks, but I'm watching $DIS for a possible short term entry. With opening of Disney Springs this week, the parks are likely going to be opening soon for phase 1.","[(like, IN), (big, JJ), (dollar, NN), (stocks, NNS), (i'm, VBP), (watching, VBG), (#dis, NNP), (possible, JJ), (short, JJ), (term, NN), (entry, NN), (opening, VBG), (disney, NN), (springs, NNS), (week, NN), (parks, NNS), (likely, RB), (going, VBG), (opening, VBG), (soon, RB), (phase, NN)]"
5,$MSIM Family taking over net worth $500 million. Huh?💸📈 #STOCKS #blockchain #Crypto #bitcoin #XRP #ico #startup $TWTR $DWDP $PNC $CVS $BABA $NIO $TGT $MER $MGM $TLRY $CHK $AMZN $COST $LULU $YUM $VRUS $LDSR $FB $SQ $VALE $YELP $TE $PSSI $JD $BZUN $XOM $BP $C $S $F $jpm $nvda $v,"[(#msim, JJ), (family, NN), (taking, VBG), (net, JJ), (worth, NN), (million, CD), (huh, NN), (💸, NNP), (📈, NNP), (#twtr, NNP), (#dwdp, NNP), (#pnc, NNP), (#cvs, NNP), (#baba, NNP), (#nio, NNP), (#tgt, NNP), (#mer, NNP), (#mgm, NNP), (#tlry, NNP), (#chk, NNP), (#amzn, NNP), (#cost, NNP), (#lulu, NNP), (#yum, NNP), (#vrus, NNP), (#ldsr, NNP), (#fb, NNP), (#sq, NNP), (#vale, NNP), (#yelp, NNP), (#te, NNP), (#pssi, NNP), (#jd, NNP), (#bzun, NNP), (#xom, NNP), (#bp, NNP), (#, #), (c, RB), (#, #), (#, #), (f, JJ), (#jpm, NNP), (#nvda, NNP), (#, #), (v, NN)]"
6,"Really hard times are ahead! Your fav restaurant and coffee shop will close, kids won't go back to school, someone you know might die from virus! This is such a big deal! Oil drillers will go bankrupt and gas will go up in long term. Really bad days ahead! $DJI $SPY $XOM $JPM $GM","[(really, RB), (hard, JJ), (times, NNS), (ahead, RB), (fav, VBP), (restaurant, JJ), (coffee, NN), (shop, NN), (close, JJ), (kids, NNS), (go, VBP), (back, RB), (school, NN), (someone, NN), (know, VBD), (might, MD), (die, VB), (virus, NN), (big, JJ), (deal, NN), (oil, NN), (drillers, NNS), (go, VBP), (bankrupt, RB), (gas, NN), (go, VBP), (long, JJ), (term, NN), (really, R

In [271]:
def get_wordnet_pos(treebank_tag):

    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return ''

In [279]:
def lemmatize_words(list_of_words):
    wordnet_lemmatizer = WordNetLemmatizer()
    lemmatized_words_pos = []
    for word_tup in list_of_words:
        current_pos = get_wordnet_pos(word_tup[1])
        if current_pos == '':
            lemmatized_words_pos.append(wordnet_lemmatizer.lemmatize(word_tup[0]))
        else:
            lemmatized_words_pos.append(wordnet_lemmatizer.lemmatize(word_tup[0],current_pos))
    return lemmatized_words_pos

In [233]:
def lemmatize_words(list_of_words):
    lemmatized = [wordnet_lemmatizer.lemmatize(t) for t in list_of_words]
    return lemmatized

In [251]:
wordnet_lemmatizer.lemmatize('were', pos='v')

'be'

In [280]:
smpl.loc[:,'Word_Token_NS_L'] = smpl.loc[:,'POS'].apply(lemmatize_words)

In [282]:
smpl[['Word_Token_NS','Word_Token_NS_L']]

,Word_Token_NS,Word_Token_NS_L
0,"[market, cap, already, higher, #dis, paid, jim, henson, ’, muppets, sure, buy, company, licensing, deal, llama, llama]","[market, cap, already, high, #dis, pay, jim, henson, ’, muppets, sure, buy, company, license, deal, llama, llama]"
1,"[#dis, move, 💪, 🏻, target, wait, play, #spy, #msft, #amzn, #fb, #sbux, #jpm, #cmg, #jnj, #baba, #nke, #ccl, #unh, #nvda, #dis, #vz, #tsla, #qqq, #msi, #wmt, #now, #nflx, #deck, #meli, #bkng, #plnt, #ba, #ccl, #wynn, #rcl, #aal]","[#dis, move, 💪, 🏻, target, wait, play, #spy, #msft, #amzn, #fb, #sbux, #jpm, #cmg, #jnj, #baba, #nke, #ccl, #unh, #nvda, #dis, #vz, #tsla, #qqq, #msi, #wmt, #now, #nflx, #deck, #meli, #bkng, #plnt, #ba, #ccl, #wynn, #rcl, #aal]"
2,"[#, #dis, #amc, california, sets, movie, theater, reopenings]","[#, #dis, #amc, california, set, movie, theater, reopenings]"
3,"[long, #jnj, #xom, #, v, #bac, #ma, #dis, #, c, #adbe, #pypl, #crm, #avgo, #abbv, #acn, #lly, #iwm, #eem, #slv, #bkln, #xlv, #xly, #db, #, #teva]","[long, #jnj, #xom, #, v, #bac, #ma, #dis, #, c, #adbe, #pypl, #crm, #avgo, #abbv, #acn, #lly, #iwm, #eem, #slv, #bkln, #xlv, #xly, #db, #, #teva]"
4,"[like, big, dollar, stocks, i'm, watching, #dis, possible, short, term, entry, opening, disney, springs, week, parks, likely, going, opening, soon, phase]","[like, big, dollar, stock, i'm, watch, #dis, possible, short, term, entry, open, disney, spring, week, park, likely, go, open, soon, phase]"
5,"[#msim, family, taking, net, worth, million, huh, 💸, 📈, #twtr, #dwdp, #pnc, #cvs, #baba, #nio, #tgt, #mer, #mgm, #tlry, #chk, #amzn, #cost, #lulu, #yum, #vrus, #ldsr, #fb, #sq, #vale, #yelp, #te, #pssi, #jd, #bzun, #xom, #bp, #, c, #, #, f, #jpm, #nvda, #, v]","[#msim, family, take, net, worth, million, huh, 💸, 📈, #twtr, #dwdp, #pnc, #cvs, #baba, #nio, #tgt, #mer, #mgm, #tlry, #chk, #amzn, #cost, #lulu, #yum, #vrus, #ldsr, #fb, #sq, #vale, #yelp, #te, #pssi, #jd, #bzun, #xom, #bp, #, c, #, #, f, #jpm, #nvda, #, v]"
6,"[really, hard, times, ahead, fav, restaurant, coffee, shop, close, kids, go, back, school, someone, know, might, die, virus, big, deal, oil, drillers, go, bankrupt, gas, go, long, term, really, bad, days, ahead, #dji, #spy, #xom, #jpm, #gm]","[really, hard, time, ahead, fav, restaurant, coffee, shop, close, kid, go, back, school, someone, know, might, die, virus, big, deal, oil, drillers, go, bankrupt, gas, go, long, term, really, bad, day, ahead, #dji, #spy, #xom, #jpm, #gm]"
7,"[#mism, going, unnoticed, silent, r, /, merger, 💸, 📈, 🛒, ✈, 🍾, #aapl, #amd, #amrn, #aal, #bac, #amzn, #msft, #ba, #abt, #tsla, #, x, #nvda, #ge, #dis, #ccl, #uber, #dal, #nflx, #fb, #, #, f, #mu, #sq, #, #zm, #xom, #gild, #roku, #ual, #jpm, #abbv, #fcx, #snap, #baba, #wfc, #rcl, #bb, #hsbc, #lyg, #teva, #, c, #oxy, #jnj, #lyft, #lk, #zm]","[#mism, go, unnoticed, silent, r, /, merger, 💸, 📈, 🛒, ✈, 🍾, #aapl, #amd, #amrn, #aal, #bac, #amzn, #msft, #ba, #abt, #tsla, #, x, #nvda, #ge, #dis, #ccl, #uber, #dal, #nflx, #fb, #, #, f, #mu, #sq, #, #zm, #xom, #gild, #roku, #ual, #jpm, #abbv, #fcx, #snap, #baba, #wfc, #rcl, #bb, #hsbc, #lyg, #teva, #, c, #oxy, #jnj, #lyft, #lk, #zm]"
8,"[gd, entertainment, technology, sets, relaunch, imagined, prepaid, ‘, dream, card, ', self, use, millennials, gen, z, consumers, drive, prepaid, gift, cards, cagr, growth, #gdet, #gdot, #axp, #pypl, #jpm]","[gd, entertainment, technology, set, relaunch, imagine, prepaid, ‘, dream, card, ', self, use, millennials, gen, z, consumer, drive, prepay, gift, card, cagr, growth, #gdet, #gdot, #axp, #pypl, #jpm]"
9,"[here's, traders, eyeing, heading, weekend, #jpm, #gdx, #aapl, #agn]","[here's, trader, eye, head, weekend, #jpm, #gdx, #aapl, #agn]"


In [283]:
fin_dict = {'stock': 134,
 'new': 90,
 'today': 76,
 'buy': 66,
 'news': 61,
 'article': 59,
 'market': 57,
 'see': 56,
 'daily': 56,
 'price': 55,
 'company': 51,
 'option': 50,
 'time': 46,
 'like': 45,
 'alert': 42,
 'dividend': 39,
 'call': 38,
 'volume': 37,
 'profit': 36,
 'share': 35,
 'invest': 0,
 'pips': 0 }

In [284]:
fin_info = set(fin_dict.keys())

In [285]:
def get_similarity_score( list_of_words, comparison_set):
    """compute similarity score i.e. number of matching words between a tweet and a predefined dictionary"""
    similarity_score = len(set(list_of_words).intersection(set(fin_info)))
    return similarity_score

In [286]:
smpl.loc[:,'Similarity_Score'] = smpl.loc[:,'Word_Token_NS_L'].apply(get_similarity_score, args=([fin_info]))

In [291]:
smpl[['text','Word_Token_NS_L','Similarity_Score']].sort_values(['Similarity_Score'], ascending = False)

,text,Word_Token_NS_L,Similarity_Score
0,Market cap is already higher than $dis paid for Jim Henson’s muppets but sure buy out a company ...,"[market, cap, already, high, #dis, pay, jim, henson, ’, muppets, sure, buy, company, license, de...",3
21,Weighing Up The Investment Potential Of Chevron Vs. Exxon Mobil https://seekingalpha.com/article...,"[weigh, investment, potential, chevron, v, exxon, mobil, #cvx, #xom, great, dividend, stock, buy...",3
10,"Today Top Flow in S&P 500 #SP500, Buy Flow and Sell Flow $NVDA $AAPL $C $T $MSFT $JWN $PEP $MA $...","[today, top, flow, &, p, buy, flow, sell, flow, #nvda, #aapl, #, c, #, #msft, #jwn, #pep, #ma, #...",2
4,"I don't like big dollar stocks, but I'm watching $DIS for a possible short term entry. With open...","[like, big, dollar, stock, i'm, watch, #dis, possible, short, term, entry, open, disney, spring,...",2
20,Collected dividends on $EXC $XOM $MPC today. #dividends #DividendInvesting #dividendincome,"[collect, dividend, #exc, #xom, #mpc, today]",2
22,#Options Flow Grid Update $XOM #OptionsTrading Exxon Mobil Shares ⬆️0.72% to $46.24 #OptionsFlow...,"[flow, grid, update, #xom, exxon, mobil, share, ⬆, ️]",1
16,Calling the bottom $tsla,"[call, bottom, #tsla]",1
15,@nikolatrevor can you please tell us more about your distribution channels? As you know $TSLA re...,"[please, tell, u, distribution, channel, know, #tsla, refused, work, dealer, globally, approach,...",1
14,"$KO in Uptrend: price expected to rise as it breaks its lower Bollinger Band on November 5, 2019...","[#ko, uptrend, price, expect, rise, break, low, bollinger, band, november, view, odds, indicator]",1
12,Sure! $PYPL is another pump-and-dump one-trick pony - any crypto is better. $KO pumped by Buffet...,"[sure, #pypl, another, pump, dump, one, trick, pony, crypto, well, #ko, pump, buffet's, lie, sug...",1


In [ ]:
# Create the bag-of-words: bow

def concat_lists(x1, x2): return x1 + x2

def create_bow(df_attribute):
    bow_dict = {}
    for ticker in df_text.ticker.unique():
        ticker_words = reduce(concat_lists, df_text[df_text.ticker == ticker][df_attribute])
        bow = Counter(ticker_words)
        bow_dict[ticker] = bow
    return bow_dict
    

bow = create_bow('Word_Token_NS_L')

# Vader sentiment

from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

df_text.loc[:,'VADER'] = df_text.loc[:,'text'].apply(sid.polarity_scores)
df_text.loc[:,'VADER_cmp'] = df_text.loc[:,'VADER'].apply(lambda x:x['compound'])


def create_fdist(tick, df_attribute):
    ticker_words = reduce(concat_lists, df_text[df_text.ticker == tick][df_attribute])
    fdist = FreqDist(ticker_words)
    return fdist

# Print the 10 most common tokens
print(bow.most_common(10))

KO_fdist = create_fdist('KO','Word_casual_NS_L')

import matplotlib.pyplot as plt
KO_fdist.plot(30,cumulative=False)
plt.show()


def reorder(df):
    """ switches the last and the penultimate attribute of a DataFrame"""
    df = df.iloc[:,np.append(np.delete(np.arange(-len(df.columns),0),-2),-2)]
    return df